In [135]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re

df = pd.read_csv('/Users/colette/Documents/GitHub/data_science/sentiment labelled sentences/imdb_labelled.txt', sep="\t", header=None)
df.columns = ["review", "rating"]

In [127]:
for column_name in df.columns:
    print("Unique values in column {} are: {}".format(column_name, df[column_name].nunique()))

Unique values in column review are: 745
Unique values in column rating are: 2


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
review    748 non-null object
rating    748 non-null int64
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [129]:
df.head()

,review,rating
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [130]:
#no missing values
df.isnull().count()

review    748
rating    748
dtype: int64

In [131]:
df.to_csv('MOIST.csv')

In [132]:
#parse words for review into positive and negative list
all_words = []

for i, j in df.iterrows():
    text = df['review'][i]
    text.lower() # Convert to lowercase
    words = re.findall("[a-z0-9']+", text) # extract the words
    all_words = all_words + words
    
sr= stopwords.words('english')
clean_tokens = all_words[:]
for word in all_words:
    if word in stopwords.words('english'):
        clean_tokens.remove(word)
freq = nltk.FreqDist(clean_tokens)
for key,val in freq.items():
    print(str(key) + ':' + str(val))
freq.plot(20, cumulative=False)

NameError: name 'stopwords' is not defined

## Attempt 1

Using only negative keywords and BernoulliNB()

In [ ]:
neg_keywords = ['bad', 'slow', 'ridiculous', 'poor', 'little', 'empt', 'predictable', 'crap'
                , 'mediocre', 'worst', 'annoy', 'bore', 'boring', 'stereotypical', 'worse', 'suck'
                , 'fail', 'cheap', 'trash', 'lack', 'mess', 'unconvinc']

for key in neg_keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    df[str(key)] = df['review'].str.contains(str(key), case=0)

In [ ]:
sns.heatmap(df.corr())

In [ ]:
data = df[neg_keywords]
target = df['rating']

In [ ]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

In [ ]:
print ('percentage wrong = {}'.format(round(243/748*100, 2)))

## Attempt 2

Using postiive keywords to see if accuracy is improved

In [ ]:
pos_keywords = ['clever', 'best', 'ador', 'appeal', 'perfect', 'beautiful', 'love', 'enjoy', 'funny', 'believ'
                ,'terrific', 'excel', 'incredible', 'fascinate', 'solid', 'hope', 'wonder', 'master', 'intelligent'
                , 'entertain', 'memorable', 'brillian', 'special', 'top']

for key in pos_keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    df[str(key)] = df['review'].str.contains(str(key), case=0)

In [ ]:
sns.heatmap(df.corr())

In [ ]:
data = df[pos_keywords]
target = df['rating']

In [ ]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

In [ ]:
print ('percentage wrong = {}'.format(round(243/748*100, 2)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

In [ ]:
print ('sensitivy = {}'.format(round(132/(132+254), 2)))
print ('specificity = {}'.format(round(346/(346+16), 2)))

## Attempt 3

Failed attempt at using both positve AND negative keywords with multinomial Naive Bayes

In [144]:
neg_keywords = ['bad', 'slow', 'ridiculous', 'poor', 'little', 'empt', 'predictable', 'crap'
                , 'mediocre', 'worst', 'annoy', 'bore', 'boring', 'stereotypical', 'worse', 'suck'
                , 'fail', 'cheap', 'trash', 'lack', 'mess', 'unconvinc']

pos_keywords = ['clever', 'best', 'ador', 'appeal', 'perfect', 'beautiful', 'love', 'enjoy', 'funny', 'believ'
                ,'terrific', 'excel', 'incredible', 'fascinate', 'solid', 'hope', 'wonder', 'master', 'intelligent'
                , 'entertain', 'memorable', 'brillian', 'special', 'top']

for key in neg_keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    df[str(key)] = df['review'].str.contains(str(key), case=0)
    
for key in pos_keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    df[str(key)] = df['review'].str.contains(str(key), case=0)

In [145]:
df_new = df.drop(['review'], axis=1)

drop_idxs = []

for idx in df_new.index:
    if not df_new.loc[idx,:].tolist():
        drop_idxs.append(idx)
df_new = df_new.drop(drop_idxs)

In [157]:
drop_index = []

for index in df_new.index:
#    if df_new.loc[index, 'rating'] == 1:
 #       if df_new.loc[index, neg_keywords].tolist():
  #          drop_index.append(index)
    if df_new.loc[index, 'rating'] == 0:
        if df_new.loc[index, pos_keywords].tolist():
            drop_index.append(index)        
        
    
#df_new = df_new.drop(drop_index)

In [158]:
df_new = df_new.drop(drop_index[:len(drop_index)//2])

In [154]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 321 to 747
Data columns (total 47 columns):
rating           374 non-null int64
bad              374 non-null bool
slow             374 non-null bool
ridiculous       374 non-null bool
poor             374 non-null bool
little           374 non-null bool
empt             374 non-null bool
predictable      374 non-null bool
crap             374 non-null bool
mediocre         374 non-null bool
worst            374 non-null bool
annoy            374 non-null bool
bore             374 non-null bool
boring           374 non-null bool
stereotypical    374 non-null bool
worse            374 non-null bool
suck             374 non-null bool
fail             374 non-null bool
cheap            374 non-null bool
trash            374 non-null bool
lack             374 non-null bool
mess             374 non-null bool
unconvinc        374 non-null bool
clever           374 non-null bool
best             374 non-null bool
ador             

In [155]:
data = df_new.drop(['rating'], axis=1)
target = df_new['rating']

# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 374 points : 114


In [159]:
print ('percentage wrong = {}'.format(round(131/567*100, 2)))

percentage wrong = 23.1


In [151]:
# Test your model with different holdout groups.
from sklearn.model_selection import train_test_split


# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.7807017543859649
Testing on Sample: 0.7689594356261023


In [160]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.61538462, 0.52631579, 0.65789474, 0.54054054, 0.64864865,
       0.62162162, 0.59459459, 0.62162162, 0.72972973, 0.72972973])